In [ ]:
#variables

In [ ]:
input_data = "car data.csv"

In [ ]:
#import libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

%matplotlib inline

In [ ]:
#Read dataset

In [ ]:
df = pd.read_csv(input_data)

In [ ]:
# data view

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
catagoical_features = [i for i in df.columns if df.dtypes[i]=='object']
catagoical_features

In [ ]:
for feature in catagoical_features:
    unique_val = df[feature].unique()
    if (len(unique_val) < 10 ):
        print(unique_val)

In [ ]:
# Checking missing values

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum()

In [ ]:
required_features = df.columns

In [ ]:
required_features = required_features.drop(['Car_Name'])
required_features

In [ ]:
final_df = df[required_features]
final_df

In [ ]:
final_df["Current_year"] = 2021
final_df

In [ ]:
final_df['Years_Used'] = final_df['Current_year'] - final_df['Year']
final_df

In [ ]:
final_df.drop(['Year','Current_year'],axis=1,inplace = True)
final_df

In [ ]:
#Data preparation

In [ ]:
final_df = pd.get_dummies(final_df , drop_first = True )
final_df

In [ ]:
#Correlation

In [ ]:
final_df.corr()

In [ ]:
sns.pairplot(final_df)

In [ ]:
corr_matrix = final_df.corr()
plt.figure(figsize = (20,20))
ax = sns.heatmap(corr_matrix, xticklabels = corr_matrix.columns, yticklabels = corr_matrix.columns,linewidths = 0.2, cmap = 'RdYlGn', annot = True)
plt.title("Correlation between variables")

In [ ]:
X = final_df[final_df.columns.drop('Selling_Price')]
Y = final_df['Selling_Price']

In [ ]:
#Feature Importance

In [ ]:
model = ExtraTreesRegressor()
model.fit(X,Y)

In [ ]:
print(model.feature_importances_)
print(list(X.columns))

In [ ]:
# plotgraph for feature importance
feature_importance = pd.Series(model.feature_importances_ , index = X.columns)
feature_importance.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2, random_state = 2)

In [ ]:
# Hyperparamers
#Randomized Search CV
# RandomizedSearchCV - best parameter finding. Its faster than gridsearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(x_test)
predictions

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [134]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)